In [8]:
import gspread as gs
gc = gs.service_account(filename='tdpcorp-ca298cc72a5d.json')
WB = 'https://docs.google.com/spreadsheets/d/11HyZN62kfRManwuUz5jNDj7wtr8YFWJz1M5zWlTcS9Q/edit#gid=0'
workbook = gc.open_by_url(WB)